In [ ]:
import sys 
sys.path.insert(0,'/Users/mike/Projects/fresnel/build')
# !pip install freud-analysis gsd
import freud
import fresnel
import gsd.hoomd
import numpy as np

In [ ]:
with gsd.hoomd.open(name="restart.gsd", mode="rb") as gsd_file:
    snap = gsd_file[0]

In [ ]:
box = freud.box.Box.from_box(snap.configuration.box)
images = snap.particles.image

unwraped_cords = box.unwrap(snap.particles.position, images)
N = snap.particles.N

In [ ]:
particle_types = snap.particles.typeid
colors = np.empty((N, 3))
# Color by typeid
colors[particle_types == 0] = [1, 0, 0]
colors[particle_types == 1] = [0, 1, 0]
colors[particle_types == 2] = [0, 0, 1]

In [ ]:
scene = fresnel.Scene()
geometry = fresnel.geometry.Sphere(scene, N=N, radius=0.2)
geometry.position[:] = snap.particles.position  # unwraped_cords
geometry.material = fresnel.material.Material(
    color=fresnel.color.linear([0.25, 0.5, 0.9]), roughness=0.8
)
geometry.material.primitive_color_mix = 1.0
geometry.color[:] = fresnel.color.linear(colors)
# box
fresnel.geometry.Box(scene, box.to_tuple())

In [ ]:
N_bonds = snap.bonds.group.shape[0]
bonds = fresnel.geometry.Cylinder(scene, N=N_bonds)
bonds.material = fresnel.material.Material(
    color=fresnel.color.linear([0.5, 0.5, 0]), roughness=0.8
)
bonds.material.primitive_color_mix = 1.0
bonds.points[:] = np.stack(
    [
        snap.particles.position[snap.bonds.group[:, 0]],
        snap.particles.position[snap.bonds.group[:, 1]],
    ],
    axis=1,
)
# Color by bond typeid
bond_ids = snap.bonds.typeid
bond_colors = np.empty((N_bonds, 3))
bond_colors[bond_ids == 0] = [0, 1, 0]
bond_colors[bond_ids == 1] = [0, 0, 1]

bonds.color[:] = np.stack(
    [fresnel.color.linear(bond_colors), fresnel.color.linear(bond_colors)], axis=1
)
bonds.radius[:] = [0.1] * N_bonds


fresnel.preview(scene, aa_level=3)